In [2]:
from pathlib import Path  

from tqdm.auto import tqdm

import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

In [3]:
stations = pd.read_csv("ghcn_mos_lookup.csv")

In [4]:
stations.head()

,Unnamed: 0,ID,LATITUDE,LONGITUDE,Station,Latitude,Longitude
0,0,USC00026180,36.9208,-111.4483,KPGA,36.9333,-111.4500
1,1,USW00003162,36.9261,-111.4478,KPGA,36.9333,-111.4500
2,2,USC00080211,29.7258,-85.0206,KAAF,29.7333,-85.0167
3,3,USW00012832,29.7333,-85.0333,KAAF,29.7333,-85.0167
4,4,USC00088782,27.7053,-82.4008,KMCF,27.8500,-82.5000


In [21]:
YEAR = 2018

In [23]:
columns = ['station', 'short_model', 'model', 'runtime', 'ftime', 'N/X', 'X/N',
           'TMP', 'DPT', 'WDR', 'WSP', 'CIG', 'VIS', 'P06', 'P12', 'POS', 'POZ',
           'SNW', 'CLD', 'OBV', 'TYP', 'Q06', 'Q12', 'T06', 'T12']
colfilt = ['station', 'runtime', 'ftime','TMP']
for f in Path('station_filter').glob(f'mav{YEAR}*.csv'):
    df = pd.read_csv(f, names = columns, parse_dates=['runtime', 'ftime'])
    df.info()
    break
    mdf = df[colfilt][df['station'].isin(stations['Station'])]
    mdf['forecast'] = (mdf['ftime'].dt.tz_localize('utc') - mdf['runtime'])
    for td, tdf in mdf.groupby('forecast'):
        tds = str(td).replace(" ", "_")
        fout = f'MOS_{YEAR}_{tds}.csv'
        mode = 'a' if Path(fout).exists() else 'w'
        tdf.to_csv(fout, mode=mode)
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269450 entries, 0 to 1269449
Data columns (total 25 columns):
 #   Column       Non-Null Count    Dtype              
---  ------       --------------    -----              
 0   station      1269450 non-null  object             
 1   short_model  1269450 non-null  object             
 2   model        1269450 non-null  object             
 3   runtime      1269450 non-null  datetime64[ns, UTC]
 4   ftime        1269450 non-null  datetime64[ns]     
 5   N/X          0 non-null        float64            
 6   X/N          298950 non-null   float64            
 7   TMP          1263150 non-null  float64            
 8   DPT          1262520 non-null  float64            
 9   WDR          1260630 non-null  float64            
 10  WSP          1260630 non-null  float64            
 11  CIG          1266300 non-null  float64            
 12  VIS          1227870 non-null  float64            
 13  P06          664620 non-null   float64    